In [17]:
!pip install mysql-connector-python

In [18]:
import mysql.connector as connection

!pip install mysql-connector-python

import mysql.connector as connection

connection = connection.connect(
    host='100.85.141.251',
    user='sru_daro',
    password='8DCzXQddp',
    database='fathstv2_new'
)

!pip install pandasql

In [19]:
import pandasql

from pandasql import sqldf

!pip install pymysql

import pymysql

db = pymysql.connect(host='100.85.141.251',
    user='sru_daro',
    password='8DCzXQddp',
    database='fathstv2_new')

In [20]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pytz
from sklearn.model_selection import train_test_split
import xgboost as Xgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [22]:
# Set the timezone to Pacific Standard Time (PST)
pst_timezone = pytz.timezone('America/Los_Angeles')

# Get the current time in PST
current_date_pst = datetime.now(pst_timezone)

# Calculate the start date for the last 60 days excluding today
start_date = current_date_pst - timedelta(days=60)
Fetch_date= start_date.strftime("%Y-%m-%d")
# Get the current time in PST
current_time_pst = datetime.now(pst_timezone)
# Calculate the start time for the last 2 hours
start_time = current_time_pst - timedelta(hours=4)
interval = start_time.strftime("%Y-%m-%d %H:%M:%S")
interval

subid_id = 71290

In [23]:
query = """SELECT 
report_date,
km.keyword_name,
(sum(attempts)) as Attempts,
(((sum(attempts)-sum(rejection))/sum(attempts))) as Acceptance_Rate,
(((sum(clicks) -sum(pushed_clicks))/ (sum(attempts)-sum(rejection)))) as Click_rate,
((sum(total_revenue)- sum(google_revenue))) as OUR_Rev
from live_ar_cache as lac
inner join keyword_master as km
on lac.keyword_id = km.id
where lac.subid_id = '{2}' AND  report_date > '{0}' #assign subID
and keyword_name in (SELECT
km.keyword_name AS keyword_name
FROM
  tracking_keywords_exposure tee
  LEFT JOIN tracking_visitors tv ON(tv.id = tee.visitor_id)
  LEFT JOIN tracking_clicks tc ON(
    tc.visitor_id = tv.id
    AND tc.keyword_id = tee.keyword_id
    AND subid_id = '{2}'                   #assign subID
    AND tc.created_date > '{1}'
  )
  LEFT JOIN live_cpc_cache lcc ON(lcc.click_id = tc.id)
  LEFT JOIN keyword_master km ON(km.id = tee.keyword_id)
  LEFT JOIN (
    SELECT
      subid_id,
      domain_id,
      subid_category_id,
      MIN(linked_date) AS linked_date,
      MAX(unlinked_date) AS unlinked_date
    FROM
      subid_category_domain_history
    GROUP BY
      subid_id,
      domain_id
  ) scda ON(
    tv.domain_id = scda.domain_id
    AND tv.created_date BETWEEN scda.linked_date
    AND scda.unlinked_date
  )
  LEFT JOIN domain_master dm ON(dm.id = tv.domain_id)
  LEFT JOIN subid_master sm ON(sm.id = scda.subid_id)
  LEFT JOIN subid_ab_profiles sap ON(
    sap.keyword_profile_id = tee.keyword_profile_id
  )
  INNER JOIN subid_category_master scm ON(scm.id = sm.subid_category_id)
WHERE
  tv.tag5 IS NOT NULL
  AND tv.tag6 IS NOT NULL
  AND sm.subid IS NOT NULL
  AND sm.id = '{2}'                                                 # assign subID
  AND tv.created_date > '{1}'
GROUP BY
  tee.keyword_id
Having
(COUNT(
    CASE WHEN tee.clicked > 0 THEN tee.visitor_id END
  )  -
   COUNT(
    CASE WHEN tee.visible_order = -1
    AND tee.clicked > 0
    AND tee.sort_order = -1 THEN tee.visitor_id END
  ) ) > 0)
  
group by  keyword_id, report_date
order by keyword_name,attempts Asc;""".format(Fetch_date,interval,subid_id)

query

df=pd.read_sql_query(query, db , parse_dates= True)
df.fillna(0, inplace=True)
query

"SELECT \nreport_date,\nkm.keyword_name,\n(sum(attempts)) as Attempts,\n(((sum(attempts)-sum(rejection))/sum(attempts))) as Acceptance_Rate,\n(((sum(clicks) -sum(pushed_clicks))/ (sum(attempts)-sum(rejection)))) as Click_rate,\n((sum(total_revenue)- sum(google_revenue))) as OUR_Rev\nfrom live_ar_cache as lac\ninner join keyword_master as km\non lac.keyword_id = km.id\nwhere lac.subid_id = '71290' AND  report_date > '2024-09-11' #assign subID\nand keyword_name in (SELECT\nkm.keyword_name AS keyword_name\nFROM\n  tracking_keywords_exposure tee\n  LEFT JOIN tracking_visitors tv ON(tv.id = tee.visitor_id)\n  LEFT JOIN tracking_clicks tc ON(\n    tc.visitor_id = tv.id\n    AND tc.keyword_id = tee.keyword_id\n    AND subid_id = '71290'                   #assign subID\n    AND tc.created_date > '2024-11-09 22:04:47'\n  )\n  LEFT JOIN live_cpc_cache lcc ON(lcc.click_id = tc.id)\n  LEFT JOIN keyword_master km ON(km.id = tee.keyword_id)\n  LEFT JOIN (\n    SELECT\n      subid_id,\n      domain_i

In [24]:
df.shape[0]

600

In [25]:
df.head()

,report_date,keyword_name,Attempts,Acceptance_Rate,Click_rate,OUR_Rev
0,2024-10-31,Best Paid Weight Loss Clinical Trials Near Me ...,1.0,1.0,1.00,0.6307
1,2024-10-09,Best Paid Weight Loss Clinical Trials Near Me ...,1.0,1.0,1.00,1.4399
2,2024-09-23,Best Paid Weight Loss Clinical Trials Near Me ...,4.0,1.0,0.75,3.5641
3,2024-10-24,Best Paid Weight Loss Clinical Trials Near Me ...,5.0,1.0,0.20,0.7378
4,2024-09-14,Best Paid Weight Loss Clinical Trials Near Me ...,5.0,1.0,0.40,3.2904


In [26]:
keywords = df['keyword_name'].unique()
keywords

array(['Best Paid Weight Loss Clinical Trials Near Me 2024',
       'free clinical trials for weight loss',
       'free weight loss surgery clinical trial',
       'Lose Weight Medication',
       'New High Paying Weight Loss Clinical Trials 2024',
       'new weight loss drug trial',
       'paid clinical trials for obesity near me',
       'Paid Medical Trials for Obesity',
       'Paid Weight Loss Clinical Trials 2024',
       'paid weight loss clinical trials near me',
       'tirzepatide weight loss trial',
       'weight loss clinical trials near me',
       'weight loss drug clinical trial', 'weight loss drug trials',
       'weight loss drug trials near me'], dtype=object)

In [27]:
# Filter unique keywords
keywords = df['keyword_name'].unique()

# Initialize a dictionary to store results for each keyword
results = {}

# Initialize an empty DataFrame to store results for each keyword
output_df = pd.DataFrame()

# Iterate through each keyword
for keyword in keywords:
    # Filter data for the current keyword
    keyword_data = df[df['keyword_name'] == keyword]
    
    # Extract features and target variables
    X = keyword_data[['Attempts', 'Acceptance_Rate', 'Click_rate']]
    y = keyword_data['OUR_Rev']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    
    dXtrain_reg = Xgb.DMatrix(X_train,y_train)
    dXtest_reg = Xgb.DMatrix(X_test,y_test)
    
    # set iteration
    n= 1000
    max_depth = 4
    learning_rate = 0.01
    
    # setting hyperparameter
    params = {"objective": "reg:squarederror", "tree_method":"hist" }
    evals = [(dXtrain_reg, "train"),(dXtest_reg, "validation")]
    
    # Train XGBoost model with early stopping
    UP_model2 = xgb.train(params=params, dtrain=dXtrain_reg, num_boost_round=n, evals=evals, verbose_eval=2, early_stopping_rounds=2)
    
    # Splitting the data into features (X) and target variable (y)
    X = keyword_data[['Attempts', 'Acceptance_Rate', 'Click_rate']]
    y = keyword_data['OUR_Rev']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Make predictions on test data
    preds_UPM2 = UP_model2.predict(xgb.DMatrix(X_test))
    
    # Convert predictions to DataFrame for easy comparison
    results = pd.DataFrame({'Actual': y_test.values, 'Predicted':preds_UPM2 ,"diff": y_test.values - preds_UPM2})

    
    n = 1000
    results = Xgb.cv(params, dXtrain_reg,num_boost_round=n,nfold=50,early_stopping_rounds=2)
    
    best_rmse = results['test-rmse-mean'].min()
    
 
    test = Xgb.DMatrix(X_test)
    predict = UP_model2.predict(test)
    r2Sq = r2_score(y_test,predict)
    
    # Set the timezone to Pacific Standard Time (PST)
    pst_timezone = pytz.timezone('America/Los_Angeles')
    # Get the current time in PST
    current_time_pst = datetime.now(pst_timezone)
    # Calculate the start time for the last 2 hours
    start_time = current_time_pst - timedelta(hours=5)
    interval = start_time.strftime("%Y-%m-%d %H:%M:%S")
    
    query_1 = """ SELECT
  km.keyword_name AS keyword_name,
  (
  COUNT(
    CASE WHEN tee.visible_order != -1 THEN tee.visitor_id ELSE NULL END
  ) 
  /
  ((
    (COUNT(tee.visitor_id) - COUNT(
      CASE WHEN tee.sort_order = -1
      AND tee.visible_order = -1
      AND tee.searched > 0 THEN tee.visitor_id ELSE NULL END
    )) - COUNT(
    CASE WHEN tee.visible_order != -1 THEN tee.visitor_id ELSE NULL END
  )
  ) )) AS Acceptance_Rate,
  
 ( (COUNT(
    CASE WHEN tee.clicked > 0 THEN tee.visitor_id END
  )  -
   COUNT(
    CASE WHEN tee.visible_order = -1
    AND tee.clicked > 0
    AND tee.sort_order = -1 THEN tee.visitor_id END
  ) ) /
  COUNT(
    CASE WHEN tee.visible_order != -1 THEN tee.visitor_id ELSE NULL END
  ))  AS OUR_Click_Rate

FROM
  tracking_keywords_exposure tee
  LEFT JOIN tracking_visitors tv ON(tv.id = tee.visitor_id)
  LEFT JOIN tracking_clicks tc ON(
    tc.visitor_id = tv.id
    AND tc.keyword_id = tee.keyword_id
    AND subid_id = '{1}'                          # assign SubID
    AND tc.created_date > '{0}'
  )
  LEFT JOIN live_cpc_cache lcc ON(lcc.click_id = tc.id)
  LEFT JOIN keyword_master km ON(km.id = tee.keyword_id)
  LEFT JOIN (
    SELECT
      subid_id,
      domain_id,
      subid_category_id,
      MIN(linked_date) AS linked_date,
      MAX(unlinked_date) AS unlinked_date
    FROM
      subid_category_domain_history
    GROUP BY
      subid_id,
      domain_id
  ) scda ON(
    tv.domain_id = scda.domain_id
    AND tv.created_date BETWEEN scda.linked_date
    AND scda.unlinked_date
  )
  LEFT JOIN domain_master dm ON(dm.id = tv.domain_id)
  LEFT JOIN subid_master sm ON(sm.id = scda.subid_id)
  LEFT JOIN subid_ab_profiles sap ON(
    sap.keyword_profile_id = tee.keyword_profile_id
  )
  INNER JOIN subid_category_master scm ON(scm.id = sm.subid_category_id)
WHERE
  tv.tag5 IS NOT NULL
  AND tv.tag6 IS NOT NULL
  AND sm.subid IS NOT NULL
  AND sm.id = '{1}'                  # assign SubID
  AND tv.created_date > '{0}'
GROUP BY
  tee.keyword_id
Having
(COUNT(
    CASE WHEN tee.clicked > 0 THEN tee.visitor_id END
  )  -
   COUNT(
    CASE WHEN tee.visible_order = -1
    AND tee.clicked > 0
    AND tee.sort_order = -1 THEN tee.visitor_id END
  ) ) > 0;
""".format(interval,subid_id)
    
    df1=pd.read_sql_query(query_1, db , parse_dates= True)
    df1.fillna(0, inplace=True)
    
    # Add the 'Attempts' parameter
    attempts = [50, 100, 150, 200]
    # Extract the common values for Acceptance_Rate and Click_rate
    acceptance_rate = df1['Acceptance_Rate'].iloc[0]
    click_rate = df1['OUR_Click_Rate'].iloc[0]
    # Create lists for Acceptance_Rate and Click_rate
    acceptance_rates = [acceptance_rate] * len(attempts)
    click_rates = [click_rate] * len(attempts)
    # Create a new DataFrame with 'Attempts', 'Acceptance_Rate', and 'Click_rate'
    result_df = pd.DataFrame({'Attempts': attempts, 'Acceptance_Rate': acceptance_rates, 'Click_rate': click_rates})
    
    # Assuming you've already trained the xg_reg model as mentioned in the previous code
    # Use the trained model to make predictions
    new_predictions = UP_model2.predict(xgb.DMatrix(result_df))

    
    # Append results to the DataFrame
   # output_df = pd.concat({'Keyword Name': keyword,'Attempts': attempts,'Predicted Rev_UPModel2': new_predictions,'Model2 Accuracy': r2Sq,'RMS-Eroor of the Model2': best_rmse,'Training data fetch from': start_date.strftime("%Y-%m-%d"),'Live data consider for prediction from': start_time.strftime("%Y-%m-%d %H:%M:%S")}, ignore_index=True).T
    #output_df

    new_data = pd.DataFrame({
    'Keyword Name': [keyword],
    'Attempts': [attempts],
    'Predicted Rev_UPModel2': [new_predictions],
    'Model2 Accuracy': [r2Sq],
    'RMS-Eroor of the Model2': [best_rmse],
    'Training data fetch from': [start_date.strftime("%Y-%m-%d")],
    'Live data consider for prediction from': [start_time.strftime("%Y-%m-%d %H:%M:%S")]
     })

    output_df = pd.concat([output_df, new_data], ignore_index=True)

# Print the DataFrame
    print(output_df)

[0]	train-rmse:127.01376	validation-rmse:71.58099
[2]	train-rmse:79.88058	validation-rmse:43.53163
[4]	train-rmse:52.87426	validation-rmse:26.83414
[6]	train-rmse:36.74068	validation-rmse:18.19334
[8]	train-rmse:25.87671	validation-rmse:12.78356
[10]	train-rmse:18.39828	validation-rmse:11.13922
[12]	train-rmse:13.17940	validation-rmse:10.92313
[13]	train-rmse:11.17768	validation-rmse:11.04452
[15:34:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[15:34:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[15:34:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[15:34:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[15:34:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/lea

C:\Users\Pranjali\Anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


                                        Keyword Name             Attempts  \
0  Best Paid Weight Loss Clinical Trials Near Me ...  [50, 100, 150, 200]   
1               free clinical trials for weight loss  [50, 100, 150, 200]   
2            free weight loss surgery clinical trial  [50, 100, 150, 200]   
3                             Lose Weight Medication  [50, 100, 150, 200]   
4   New High Paying Weight Loss Clinical Trials 2024  [50, 100, 150, 200]   
5                         new weight loss drug trial  [50, 100, 150, 200]   
6           paid clinical trials for obesity near me  [50, 100, 150, 200]   

                         Predicted Rev_UPModel2  Model2 Accuracy  \
0  [24.935255, 58.138844, 163.59862, 163.59862]         0.998141   
1   [20.540487, 21.565512, 21.565512, 76.20525]         0.991527   
2   [4.955351, 11.978971, 11.947662, 27.796976]         0.996588   
3  [1.9452823, 1.9452823, 1.9452823, 1.9452823]         0.500994   
4   [4.779656, 19.905231, 19.905231, 19.905

In [28]:
 pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [29]:
#df = pd.DataFrame(data)
import xlsxwriter
# Create an Excel writer object
#excel_writer = pd.ExcelWriter('output_data.xlsx', engine='xlsxwriter')
#excel_writer = pd.ExcelWriter('C:/Users/Pranjali/OneDrive/Documents/Python Scripts/output_data.xlsx', engine='xlsxwriter')
import pandas as pd
#excel_writer = pd.ExcelWriter('output_data.xlsx', engine='xlsxwriter')-- For office use only
excel_writer = pd.ExcelWriter('C:/Users/Pranjali/OneDrive/Documents/Python Scripts/output_data.xlsx', engine='xlsxwriter')




# Convert the DataFrame to an Excel file
output_df.to_excel(excel_writer, index=False)


# Save the Excel file
#excel_writer.Save()
excel_writer.close()

In [30]:
#connection.close()